# Przypisanie sentymentu dla jak największej liczby słów

1. Import danych z projektu NAWL - Nencki Affective Word List z oznaczeniem sentymentu
2. Przypisanie sentymentu w oparciu o podstawowe słowa
3. Wykorzystanie embedingów do przypisania sentymentu na podstawie sąsiadujących słów

In [2]:
import re
import pandas as pd
import numpy as np
import funcs 
import importlib
import unicodecsv as csv
import pickle
import random
import gensim
from gensim import corpora, models
from timeit import default_timer as timer
from datetime import datetime, timedelta
import nltk
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
import spacy
from collections import Counter
import pyLDAvis
import pyLDAvis.gensim
from sklearn.model_selection import train_test_split
import scipy

importlib.reload(funcs)

<module 'funcs' from 'D:\\_projekty\\praca\\funcs.py'>

In [3]:
%time dane = pd.read_csv('00_dane/dane_cleanest.csv',  sep='|', encoding='UTF-8')
%time dane['speech_rawest'] = dane['speech_rawest'].apply(lambda x: re.sub(r'[\W\d]',' ', x))
%time dane['speech_rawest'] = dane['speech_rawest'].apply(lambda x: [word for word in x.split()])

KeyboardInterrupt: 

NameError: name 'dane' is not defined

NameError: name 'dane' is not defined

In [33]:
dane.tail()

,nr,date_,speech_rawer,full_name,party_final,speech_rawest,topic_LDA_BOW,topic_LDA_TF,opis_LDA_BOW,opis_LDA_TF
158315,158874,31-08-2011,Sojuszowi Lewicy Demokratycznej bardzo zależy...,Romuald Ajchler,Sojusz Lewicy Demokratycznej,"[sojusz, lewica, demokratyczny, zależeć, ziemi...",26.0,8.0,posiedzenie,posiedzenie
158316,158875,31-08-2011,Panie Przewodniczący Krajowej Rady Sądownictw...,Stanisław Rydzoń,Sojusz Lewicy Demokratycznej,"[przewodniczący, krajowy, rada, sądownictwo, z...",16.0,13.0,wymiar sprawiedliwości,przepisy
158317,158876,31-08-2011,Mam pytanie do pana ministra. Panie ministrze...,Stanisław Rydzoń,Sojusz Lewicy Demokratycznej,"[mieć, pytanie, minister, minister, mieć, doty...",28.0,2.0,handel,inwestycje
158318,158877,31-08-2011,Pani Minister! Bardzo dużo mówi się o szkolni...,Stanisław Stec,Sojusz Lewicy Demokratycznej,"[minister, mówić, szkolnictwo, zawodowy, dosto...",7.0,2.0,NaN,inwestycje
158319,158878,31-08-2011,Szanowna Pani Minister! Bardzo ważnym zadanie...,Tadeusz Tomaszewski,Sojusz Lewicy Demokratycznej,"[szanowny, minister, ważny, zadanie, władza, p...",25.0,11.0,samorządy,rodzina


## 1. Wczytanie listy słów z NAWL

Parametry wg których przypisano kategorie zmieniono aby zwiększyć liczbę przypisań. Usunięto grupę 'neutral'.

In [4]:
NAWL = pd.read_csv('00_dane/nawl-analysis_2.csv', encoding='UTF-8')
NAWL.head()

,word,category,mean Happiness,mean Anger,mean Sadness,mean Fear,mean Disgust,distance to H,distance to A,distance to S,distance to F,distance to D,distance to N
0,tropiki,H,4.518518,1.703704,1.592593,2.074074,1.629630,3.637347,7.251989,7.312897,6.880706,7.236647,4.675840
1,łączyć,H,3.814815,1.666667,1.925926,1.777778,1.666667,4.161421,6.728334,6.476909,6.615627,6.693552,3.628548
2,bogaty,H,4.592593,2.370370,1.703704,1.703704,1.925926,3.826875,6.801012,7.372475,7.404916,7.217664,4.897808
3,rozkoszny,H,5.296296,1.407407,1.481481,1.555556,1.370370,2.366185,7.513857,7.449380,7.387380,7.536883,4.835511
4,nabytek,H,4.555555,1.666667,1.666667,1.555556,1.444444,3.227771,7.022504,6.998906,7.096976,7.198279,4.325238


In [102]:
print('Lista zawiera {} słów.'.format(len(NAWL)))

Lista zawiera 2902 słów.


Analiza wczytanych danych

In [7]:
NAWL.groupby('category')["word"].count()
#NAWL.groupby('category')["mean Anger"].mean()

category
A     141
D      59
F     216
H    1193
S      93
U    1200
Name: word, dtype: int64

Klasa "U" oznacza brak klasyfikacji. Mamy więc mało przypisanych słów :/

Inne:
- Happiness
- Anger
- Sadness
- Fear
- Neutral (zmia=eniono parametry tak, aby się nie dopsiywało)
- Disgust


Przypiszmy mimo wszystko flagę, do której odległość jest najmniejsza.

In [5]:
NAWL['new_flag']=''
for i in range(len(NAWL)):
    a = NAWL.iloc[i]
    m = min(a['distance to H'], a['distance to A'], a['distance to S'], a['distance to F'], a['distance to D'])
    if m == a['distance to H']:
        NAWL.loc[i, 'new_flag']='H'
    elif m == a['distance to S']:
        NAWL.loc[i, 'new_flag']='S'
    elif m == a['distance to F']:
        NAWL.loc[i, 'new_flag']='F'
    elif m == a['distance to D']:
        NAWL.loc[i, 'new_flag']='D'
    elif m == a['distance to A']:
        NAWL.loc[i, 'new_flag']='A'

In [6]:
NAWL.groupby('new_flag')["word"].count()

new_flag
A     322
D     149
F     465
H    1740
S     226
Name: word, dtype: int64

In [7]:
NAWL[(NAWL['category']!=NAWL['new_flag']) & (NAWL['category']!='U')]

,word,category,mean Happiness,mean Anger,mean Sadness,mean Fear,mean Disgust,distance to H,distance to A,distance to S,distance to F,distance to D,distance to N,new_flag


In [8]:
NAWL.drop('distance to N', axis=1)

,word,category,mean Happiness,mean Anger,mean Sadness,mean Fear,mean Disgust,distance to H,distance to A,distance to S,distance to F,distance to D,new_flag
0,tropiki,H,4.518518,1.703704,1.592593,2.074074,1.629630,3.637347,7.251989,7.312897,6.880706,7.236647,H
1,łączyć,H,3.814815,1.666667,1.925926,1.777778,1.666667,4.161421,6.728334,6.476909,6.615627,6.693552,H
2,bogaty,H,4.592593,2.370370,1.703704,1.703704,1.925926,3.826875,6.801012,7.372475,7.404916,7.217664,H
3,rozkoszny,H,5.296296,1.407407,1.481481,1.555556,1.370370,2.366185,7.513857,7.449380,7.387380,7.536883,H
4,nabytek,H,4.555555,1.666667,1.666667,1.555556,1.444444,3.227771,7.022504,6.998906,7.096976,7.198279,H
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2897,guz,U,1.148148,3.666667,4.703704,4.666666,3.703704,9.322883,7.693601,6.701199,6.865419,7.687548,S
2898,asekuracyjny,U,3.259259,1.592593,1.629630,2.925926,1.666667,5.050983,6.919656,6.853235,5.591976,6.836249,H
2899,ambona,U,2.074074,2.444444,1.925926,2.148148,2.296296,6.417684,6.330169,6.712734,6.486877,6.465244,A
2900,bezsensowny,U,1.740741,3.444444,3.037037,2.666667,2.407408,7.320939,6.228213,6.572936,7.014468,7.238194,A


## 2. Przypisanie sentymentu słów

### 2.1 Budowa słownika wszystkich słów

In [9]:
dict_sp = {}
for sp in dane['speech_rawest']:
    for w in sp:
        if w not in dict_sp.keys():
            dict_sp[w]=0
        dict_sp[w]+=1

NameError: name 'dane' is not defined

In [9]:
print('Liczba unikalnych słów w słowniku wynosi {}.'.format(len(dict_sp)))

Liczba unikalnych słów w słowniku wynosi 152498.


In [111]:
df_sp = pd.DataFrame(list(dict_sp.items()), columns=['word', 'counter'])
df_sp.head()

,word,counter
0,chcieć,205353
1,zdanie,46410
2,powiedzieć,114946
3,temat,36539
4,skutek,31434


In [20]:
len(df_sp)

152498

### 2.2 Dogranie parametrów

In [112]:
df_sp = pd.merge(df_sp, NAWL, left_on='word', right_on='word', how='left' )

In [113]:
df_sp[200:210]

,word,counter,category,mean Happiness,mean Anger,mean Sadness,mean Fear,mean Disgust,distance to H,distance to A,distance to S,distance to F,distance to D,distance to N,new_flag
200,placówka,5440,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
201,zagraniczny,27255,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
202,wrócić,5807,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
203,kraj,81373,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
204,śmierć,7324,U,1.296296,3.814815,5.888889,6.000000,3.148148,10.201947,8.788602,7.118899,7.020968,9.272644,8.381661,F
205,odmówić,1955,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
206,sprawie,42793,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
207,uważać,48841,U,2.153846,1.923077,1.423077,2.500000,1.307692,5.717540,6.015101,6.538665,5.321966,6.624956,2.721546,F
208,kierować,19754,H,4.500000,1.692308,1.653846,2.576923,1.307692,3.921665,7.148361,7.184477,6.268880,7.454568,4.439679,H
209,przesłanka,7073,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Ile mamy dogranych?

In [114]:
len(df_sp[pd.isnull(df_sp['category'])==False])

2694

In [115]:
pickle.dump(df_sp, open('df_sp.p', 'wb'))

## 3. Dogranie embeddingów

In [9]:
df_sp = pickle.load(open('df_sp.p','rb'))

In [11]:
df_sp.head()

,word,counter,category,mean Happiness,mean Anger,mean Sadness,mean Fear,mean Disgust,distance to H,distance to A,distance to S,distance to F,distance to D,distance to N,new_flag
0,chcieć,205353,H,4.333334,1.777778,2.037037,2.148148,1.592593,4.030499,7.033235,6.847620,6.729651,7.201385,4.369833,H
1,zdanie,46410,H,2.962963,1.222222,1.185185,1.444444,1.148148,4.453825,6.560185,6.583048,6.347609,6.579155,2.303450,H
2,powiedzieć,114946,U,3.000000,1.346154,1.538462,2.307692,1.192308,5.129413,6.853925,6.679800,5.946053,6.967851,2.799579,H
3,temat,36539,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,skutek,31434,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



### 3.1 Pobranie modelu dla języka polskiego

In [1]:
import fasttext.util
#fasttext.util.download_model('pl', if_exists='ignore')  
ft = fasttext.load_model('cc.pl.300.bin')

### 3.2 Słowniki pomocnicze

In [10]:
NAWL_vec_dict = {}
for w in NAWL["word"]:
    NAWL_vec_dict[w] = ft.get_word_vector(w)
corp_vec_dict = {}
for w in df_sp["word"]:
    corp_vec_dict[w] = ft.get_word_vector(w)    

In [13]:
pickle.dump(NAWL_vec_dict, open('nawl.p', 'wb'))
pickle.dump(corp_vec_dict, open('corp_vec.p', 'wb'))
                        

### 3.3 Przypisanie sentymentu na podstawie pięciu najbliższych słów z listy NAWL

In [23]:
neigh_cnt =5 #parametr z ilu słów korzystamy
max_dist = 0.8

for word in df_sp['word'][pd.isnull(df_sp['new_flag'])==True]:

    word_vec = corp_vec_dict[word]
    word_dict = {}
    neigh_df = pd.DataFrame(columns=['distance to H','distance to A','distance to S','distance to F','distance to D'])
    
    #obliczamy odległość do każdego słowa z NAWL
    for k in NAWL_vec_dict.keys():
        word_dict[k] = scipy.spatial.distance.cosine(word_vec, NAWL_vec_dict[k])

    #dla 'neigh_cnt' najbliższym (pod warunkiem, że odległość nie jest bardzo duża)
    for neighbor, dist in sorted(word_dict.items(), key=lambda item: item[1])[:neigh_cnt]:
        if dist<=max_dist:
            neigh_df = neigh_df.append(NAWL[['distance to H','distance to A','distance to S','distance to F','distance to D']][NAWL['word']==neighbor] )

    if len(neigh_df>0):
        df_sp.loc[df_sp.word==word, 'distance to H'] = neigh_df.mean()['distance to H']
        df_sp.loc[df_sp.word==word, 'distance to A'] = neigh_df.mean()['distance to A']
        df_sp.loc[df_sp.word==word, 'distance to S'] = neigh_df.mean()['distance to S']
        df_sp.loc[df_sp.word==word, 'distance to F'] = neigh_df.mean()['distance to F']
        df_sp.loc[df_sp.word==word, 'distance to D'] = neigh_df.mean()['distance to D']

        m = neigh_df.mean().min()
        if m == neigh_df.mean()['distance to H']:
            df_sp.loc[df_sp.word==word, 'new_flag']='H'
        elif m == neigh_df.mean()['distance to S']:
            df_sp.loc[df_sp.word==word, 'new_flag']='S'
        elif m == neigh_df.mean()['distance to F']:
            df_sp.loc[df_sp.word==word, 'new_flag']='F'
        elif m == neigh_df.mean()['distance to D']:
            df_sp.loc[df_sp.word==word, 'new_flag']='D'
        elif m == neigh_df.mean()['distance to A']:
            df_sp.loc[df_sp.word==word, 'new_flag']='A'

In [24]:
print("Uzupełniony sentyment dla {} z {} słów.".format(len(df_sp[pd.isnull(df_sp['new_flag'])==False]), len(df_sp)))

Uzupełniony sentyment dla 152021 z 152498 słów.


In [25]:
pickle.dump(df_sp, open('df_sp.p', 'wb'))